In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras import datasets, layers, models, preprocessing
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import numpy as np
import os
from tqdm import tqdm

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# project parameters
IMAGE_SIZE = 256
EPOCHS = 15
BATCH_SIZE = 32
ROOT = "./src/"#"../input/bee-vs-wasp/kaggle_bee_vs_wasp/"

## Image processing

First, I will load the images, resize the image and normalize them. Then, I will split the dataset into two sets, the training and test set.  

In [ ]:
df = pd.read_csv(ROOT+"labels.csv")
df.head()

In [ ]:
# # we only keep photo with a quality equals to 1
# df = df.query('photo_quality == 1').reset_index(drop=True)
# df.head()

In [ ]:
# change \ in path to /
for idx in tqdm(df.index):
    df.loc[idx,'path']=df.loc[idx,'path'].replace('\\', '/')

df.head()

In [ ]:
# number of image in every labels
number_per_label = df['label'].value_counts()
print(number_per_label)

In [ ]:
plt.hist(list(number_per_label.index), bins=[k for k in range(len(number_per_label.values)+1)],weights=number_per_label.values, rwidth=0.9, align='left')
plt.xlabel('Type of image')
plt.ylabel('Number of image')
plt.show()

The dataset already includes the training data, validation data and final validation data.

Below, I split the dataframe regarding those properties.

In [ ]:
train_df = df.query('is_validation == 0 & is_final_validation == 0').reset_index(drop=True)
val_df = df.query('is_validation == 1').reset_index(drop=True)
test_df = df.query('is_final_validation == 1').reset_index(drop=True)

In [ ]:
print("Taining set : \n",train_df['label'].value_counts())
print("\nValidation set : \n",val_df['label'].value_counts())
print("\nFinal validation set : \n",test_df['label'].value_counts())

Our datasets are not balanced, there are way more wasp images than 'other' images. 
In a first attempt, we will consider the dataset balanced. 

Let's plot some images

In [ ]:
# let's see some bee images
plt.figure()
for i in range(1,10):
    im = Image.open(ROOT + df.query('label=="bee"')['path'].reset_index(drop=True)[i-1],mode="r")
    im = np.array(im)/255
    plt.subplot(3,3,3*(i//3)+i%3)
    plt.axis('off')
    plt.imshow(im)

In [ ]:
# let's see some wasp images
plt.figure()
for i in range(1,10):
    im = Image.open(ROOT + df.query('label=="wasp"')['path'].reset_index(drop=True)[i-1],mode="r")
    im = np.array(im)
    plt.subplot(3,3,3*(i//3)+i%3)
    plt.axis('off')
    plt.imshow(im)

In [ ]:
# let's see some other insect images
try:
    plt.figure()
    for i in range(1,10):
        im = Image.open(ROOT + df.query('label=="insect"')['path'].reset_index(drop=True)[i-1],mode="r")
        im = np.array(im)
        plt.subplot(3,3,3*(i//3)+i%3)
        plt.axis('off')
        plt.imshow(im)
except:
    print("No image")

In [ ]:
# let's see some other images
try :
    plt.figure()
    for i in range(1,10):
        im = Image.open(ROOT + df.query('label=="other"')['path'].reset_index(drop=True)[i-1],mode="r")
        im = np.array(im)
        plt.subplot(3,3,3*(i//3)+i%3)
        plt.axis('off')
        plt.imshow(im)
except:
    print("No image")

In order to train a machine learning algorithm on those data, I will first create the 3 datasets :
- training set
- validation set
- final validation set.


The datasets will be composed of the processed images and the labels.

In [ ]:
# data loader
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
val_fin_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
            train_df, 
            directory=ROOT, 
            x_col='path', 
            y_col='label',
            classes=['bee','wasp','insect','other'],
            target_size=(IMAGE_SIZE, IMAGE_SIZE), 
            batch_size=BATCH_SIZE, 
            shuffle=True, 
            seed=123
        )
validation_generator = val_datagen.flow_from_dataframe(
            val_df, 
            directory=ROOT, 
            x_col='path', 
            y_col='label',
            classes=['bee','wasp','insect','other'],
            target_size=(IMAGE_SIZE, IMAGE_SIZE), 
            batch_size=BATCH_SIZE, 
            shuffle=True, 
            seed=123
        )
validation_finale_generator = val_fin_datagen.flow_from_dataframe(
            test_df, 
            directory=ROOT, 
            x_col='path', 
            y_col='label',
            classes=['bee','wasp','insect','other'],
            target_size=(IMAGE_SIZE, IMAGE_SIZE), 
            batch_size=BATCH_SIZE, 
            shuffle=True, 
            seed=123
        )

In [ ]:
ROTATION = 0.2
ZOOM = 0.2
CONTRAST = 0.2

data_augmentation = tf.keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", input_shape=(IMAGE_SIZE,IMAGE_SIZE,3)),
    layers.experimental.preprocessing.RandomRotation(ROTATION),
    layers.experimental.preprocessing.RandomZoom(ZOOM),
    #layers.experimental.preprocessing.RandomContrast([1-CONTRAST, 1+CONTRAST])
  ]
)

In [ ]:
model = models.Sequential(data_augmentation)
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.summary()

In [ ]:
layers.Dropout(0.2)
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
layers.Dropout(0.2)
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(4,activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy'])

In [ ]:
tf.debugging.set_log_device_placement(True)

# Place tensors on the CPU
with tf.device('/:GPU:0'):    
    history = model.fit_generator(
        train_generator, 
        steps_per_epoch=200,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=100
    )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']


plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(validation_finale_generator, verbose=2)

### Confusion matrix

In [ ]:
confusion_matrix = tf.math.confusion_matrix(
    validation_finale_generator.classes, np.argmax(model.predict(validation_finale_generator), axis=1)
)
print(confusion_matrix)

In [ ]:
print(np.diag(confusion_matrix))

In [ ]:
good_prediction = np.sum(np.diag(confusion_matrix))
wrong_prediction = np.sum(confusion_matrix) - good_prediction

print("Good prediction : ", good_prediction)
print("Wrong prediction : ", wrong_prediction)

### AUC-ROC score and ROC-curve


## Test of this model on some images

In [ ]:
def get_image(df,path,image_size=IMAGE_SIZE):
    """
    This function takes a dataframe df and returns the image with path 'path' after processing it  
    """
    idx = df['path'][df['path']==path].index.tolist()[0]
    im = np.array(Image.open(ROOT+df['path'][idx], mode='r'))/255
    label = df['label'][idx]
    try:
        im = tf.image.resize(im, (image_size,image_size), method=tf.image.ResizeMethod.BILINEAR, preserve_aspect_ratio=False, antialias=False, name=None)
    except :
        # if the image is a grayscale
        im_3d = np.zeros((im.shape[0],im.shape[1],3))
        for i in range(im.shape[0]):
            for j in range(im.shape[1]):
                im_3d[i,j,:] = im[i,j]
        im = tf.image.resize(im_3d, (image_size,image_size), method=tf.image.ResizeMethod.BILINEAR, preserve_aspect_ratio=False, antialias=False, name=None)
    return im, label

In [ ]:
def prediction(model,df,list_path):
    """
    This functions takes the trained model and a list of paths and returns
    the list of predicted label for the images corresponding th the paths.
    It plots every images with the prediction
    """
    label = ["bee","wasp","insect","other"]
    plt.figure()
    images = []
    list_true = []
    list_prediction = []
    for path in list_path:
        im, lbl = get_image(df,path)
        images.append(im)
        list_true.append(lbl)        
    for lbl in np.argmax(model.predict(np.array(images)),axis=1):
        list_prediction.append(label[lbl])
        
    for i in range(len(list_true)):
        if i >= 9:
            break
        plt.subplot(3,3,3*((i+1)//3)+(i+1)%3)
        plt.axis('off')
        plt.imshow(images[i])
        plt.title(list_prediction[i] + " - " + list_true[i])
    
    plt.show()
        
    return list_true,list_prediction

In [ ]:
prediction(model,df,['bee1/1240800_e5f2b40032_n.jpg',
                     'other_noinsect/521021.jpg',
                     'other_noinsect/521697.jpg',
                     'other_insect/5826066_3071dcf48f_n.jpg',
                     'other_insect/7091961939_f90294ee2e_w.jpg'
                    ]
          )

## Conclusion of the CNN

Accuracy : 80%
